## <b>3.3 PREPROCESSAMENT I ANÀLISI DE DADES</b>

### <b>3.3.4 Enginyeria de variables avançada</b>

#### <b>3.3.4.1 Objectiu i setup</b>

In [4]:
# ============================================================
# 3.3.4.1 ENGINYERIA DE VARIABLES AVANÇADA
# SETUP INICIAL I DEFINICIÓ DE DATASETS PER MODEL
# ============================================================
# En aquest script inicio la fase d’enginyeria de variables avançada.
# El meu objectiu aquí és preparar, de manera neta i traçable, els
# datasets base que utilitzaré posteriorment en cada tipus de model.
#
# Concretament:
#   - Carrego el dataset final preprocessat del pipeline anterior.
#   - Defineixo clarament quines variables entren a cada model:
#       * Freqüència: probabilitat de sinistre anual (Has_claims_year).
#       * Severitat: cost del sinistre, només quan hi ha sinistre.
#       * Ràtio econòmica: Claims_to_premium_ratio com a target independent.
#   - Reconstrueixo l’any de pòlissa i el split temporal train/test.
#   - Creo datasets base sense leakage des de l’origen.
#   - Verifico duplicats, nuls i coherència abans d’exportar.
# ============================================================

import os
import numpy as np
import pandas as pd

# Ajusto opcions de visualització per facilitar inspecció manual
pd.set_option("display.max_columns", 200)
pd.set_option("display.width", 140)

# ------------------------------------------------------------
# Funció auxiliar: control de columnes duplicades
# ------------------------------------------------------------
# Aquesta funció la faig servir com a mecanisme defensiu.
# Si en algun punt s’han colat columnes duplicades (per merges,
# concatenacions o errors previs), aquí les detecto i les elimino.
def ensure_no_duplicate_columns(df_in: pd.DataFrame, name: str) -> pd.DataFrame:
    dup_mask = df_in.columns.duplicated()
    if dup_mask.any():
        dup_cols = df_in.columns[dup_mask]
        print(f"\nATENCIÓ: columnes duplicades detectades a {name}: {list(dup_cols)}")
        df_out = df_in.loc[:, ~dup_mask].copy()
        print(f"   → Eliminades {df_in.shape[1] - df_out.shape[1]} columnes duplicades.")
        return df_out
    else:
        print(f"\n{name}: sense columnes duplicades ({df_in.shape[1]} columnes).")
        return df_in

# ------------------------------------------------------------
# 0) Càrrega del dataset canònic
# ------------------------------------------------------------
# Carrego el dataset final del pipeline (ja net, ordenat i validat).
data_path = "transformed_motor_insurance.csv"

df = pd.read_csv(data_path, sep=",", encoding="utf-8")
print("Dataset carregat:", df.shape)

# ------------------------------------------------------------
# 1) Definició de variables clau
# ------------------------------------------------------------
# Defineixo explícitament noms de columnes importants
# per evitar errors de string repetits més endavant.
id_col = "ID"

target_freq  = "Has_claims_year"
target_sev   = "Cost_claims_year"
target_ratio = "Claims_to_premium_ratio"

# Predictors ex ante per a FREQÜÈNCIA
# Aquí m’asseguro de no incloure cap variable ex post ni derivada del cost.
freq_features_base = [
    "Driver_age", "Licence_age", "Vehicle_age",
    "Has_lapse", "Policy_duration",
    "Second_driver", "Area", "Type_risk", "Type_fuel",
    "Has_claims_history",
    "Value_vehicle", "Power", "Premium",
    "Seniority", "Policies_in_force",
    "Distribution_channel"
]

# Predictors ex ante per a SEVERITAT (només quan hi ha sinistre)
# Important: no incloc cap variable que utilitzi el cost del sinistre
# ni cap ràtio econòmica derivada del target.
sev_features_base = [
    "Vehicle_age", "Value_vehicle", "Power",
    "Type_risk", "Area", "Policy_duration",
    "Weight", "Cylinder_capacity", "Premium", "Length"
]

# Variables de control i traçabilitat
# Aquestes no són predictors de negoci, però m’interessa tenir-les
# per auditoria, diagnòstic i control de qualitat.
control_features = [
    "Licence_incoherent_flag",
    "Policy_incoherent_flag",
    "Lapse_incoherent_flag",
    "Length_missing_flag"
]

# ------------------------------------------------------------
# 2) Any de pòlissa i partició temporal
# ------------------------------------------------------------
# Reconstrueixo l’any de pòlissa a partir de Date_last_renewal
# per assegurar coherència amb l’EDA i amb la validació temporal.
date_col = "Date_last_renewal"
df[date_col] = pd.to_datetime(df[date_col], errors="coerce")

df["Policy_year"] = df[date_col].dt.year
df = df.dropna(subset=["Policy_year"]).copy()
df["Policy_year"] = df["Policy_year"].astype(int)

# Defineixo el split temporal exactament igual que abans:
#   train: anys < 2018
#   test : any 2018
df["set_type"] = np.where(df["Policy_year"] < 2018, "train", "test")

print("\nDistribució train/test:")
print(df["set_type"].value_counts())

# ------------------------------------------------------------
# 3) DATASET BASE DE FREQÜÈNCIA (sense leakage)
# ------------------------------------------------------------
# Construeixo el dataset base per al model de freqüència.
# Inclou:
#   - identificador
#   - info temporal mínima
#   - predictors ex ante
#   - target de freqüència
#   - flags de control
freq_vars = (
    [id_col, "Policy_year", "set_type"] +
    freq_features_base +
    [target_freq] +
    control_features
)

# Elimino possibles duplicats de la llista de columnes
freq_vars = list(dict.fromkeys(freq_vars))
# Em quedo només amb les columnes que realment existeixen al dataframe
freq_vars = [v for v in freq_vars if v in df.columns]

df_freq_base = df[freq_vars].copy()
df_freq_base = ensure_no_duplicate_columns(df_freq_base, "df_freq_base")

# ------------------------------------------------------------
# 4) DATASET BASE DE SEVERITAT (només sinistres, sense leakage)
# ------------------------------------------------------------
# Per a severitat, em quedo només amb registres amb:
#   - Has_claims_year = 1
#   - cost positiu i informat
mask_sev = (
    (df[target_freq] == 1) &
    (df[target_sev].notna()) &
    (df[target_sev] > 0)
)

df_sev = df.loc[mask_sev].copy()

sev_vars = (
    [id_col, "Policy_year", "set_type"] +
    sev_features_base +
    [target_sev] +
    control_features
)

sev_vars = list(dict.fromkeys(sev_vars))
sev_vars = [v for v in sev_vars if v in df_sev.columns]

df_sev_base = df_sev[sev_vars].copy()
df_sev_base = ensure_no_duplicate_columns(df_sev_base, "df_sev_base")

# ------------------------------------------------------------
# 5) DATASET BASE DE RÀTIO ECONÒMICA (model independent)
# ------------------------------------------------------------
# Aquest dataset el preparo per separat, ja que la ràtio econòmica
# és un target diferent i no condicionat a tenir sinistre.
ratio_vars = (
    [id_col, "Policy_year", "set_type"] +
    freq_features_base +
    [target_ratio] +
    control_features
)

ratio_vars = list(dict.fromkeys(ratio_vars))
ratio_vars = [v for v in ratio_vars if v in df.columns]

df_ratio_base = df[ratio_vars].copy()
df_ratio_base = ensure_no_duplicate_columns(df_ratio_base, "df_ratio_base")

# ------------------------------------------------------------
# 6) Comprovació ràpida de nuls
# ------------------------------------------------------------
# Faig una comprovació ràpida de nuls per detectar problemes evidents
# abans de passar a la fase de modelització.
def quick_missing_report(df_in, name):
    missing = df_in.isna().sum()
    missing = missing[missing > 0].sort_values(ascending=False)
    print(f"\nNuls a {name}:")
    if missing.empty:
        print("  Cap nul no estructural.")
    else:
        print(missing)

quick_missing_report(df_freq_base,  "df_freq_base")
quick_missing_report(df_sev_base,   "df_sev_base")
quick_missing_report(df_ratio_base, "df_ratio_base")

# ------------------------------------------------------------
# 7) Exportació datasets base
# ------------------------------------------------------------
# Exporto tots els datasets base a una carpeta específica
# perquè quedin llestos per a la fase de modelització.
os.makedirs("data/processed", exist_ok=True)

df_freq_base.to_csv("data/processed/df_freq_base.csv", index=False)
df_sev_base.to_csv("data/processed/df_sev_base.csv", index=False)
df_ratio_base.to_csv("data/processed/df_ratio_base.csv", index=False)

print("\n3.3.4.1 complet.")
print(" - df_freq_base: predictors ex ante, sense leakage.")
print(" - df_sev_base : severitat condicionada a sinistre, sense variables derivades del target.")
print(" - df_ratio_base: dataset específic per al model de ràtio econòmica.")


Dataset carregat: (105555, 44)

Distribució train/test:
set_type
train    69740
test     35815
Name: count, dtype: int64

df_freq_base: sense columnes duplicades (24 columnes).

df_sev_base: sense columnes duplicades (18 columnes).

df_ratio_base: sense columnes duplicades (24 columnes).

Nuls a df_freq_base:
Policy_duration    70408
dtype: int64

Nuls a df_sev_base:
Policy_duration    11936
dtype: int64

Nuls a df_ratio_base:
Policy_duration    70408
dtype: int64

3.3.4.1 complet.
 - df_freq_base: predictors ex ante, sense leakage.
 - df_sev_base : severitat condicionada a sinistre, sense variables derivades del target.
 - df_ratio_base: dataset específic per al model de ràtio econòmica.


#### <b>3.3.4.2 Transformacions de variables numèriques</b>

In [6]:
# ============================================================
# 3.3.4.2 TRANSFORMACIONS DE VARIABLES NUMÈRIQUES
# ============================================================
# En aquest script faig la part d’enginyeria numèrica avançada pensada
# específicament per a models actuarials i de Machine Learning.
# Treballo sempre a partir dels dataframes base ja validats, sense
# tocar el dataset original del pipeline.
#
# El que faig en aquest pas és:
#  1) Assegurar-me que les variables clau són realment numèriques
#  2) Crear transformacions logarítmiques per variables amb cues llargues
#  3) Aplicar capping conservador (winsorització) als extrems
#  4) Estandarditzar variables per a models basats en gradient
#  5) Discretitzar variables per facilitar models lineals/GLM
#  6) Fer una imputació final molt simple per evitar NaN residuals
#  7) Guardar els datasets transformats llestos per modelització
#
# Nota important:
#  - No faig cap reemplaçament de coma/punt decimal perquè aquí ja
#    estic treballant amb el dataset canònic amb decimals en punt.
# ============================================================

# Importo les llibreries necessàries
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler    # Per estandarditzar variables contínues
from sklearn.preprocessing import KBinsDiscretizer  # Per fer binning en quantils

# ------------------------------------------------------------
# 0) Càrrega dels datasets base
# ------------------------------------------------------------
# Carrego els tres datasets base creats al pas anterior,
# cadascun pensat per a un tipus de model diferent.
df_freq  = pd.read_csv("data/processed/df_freq_base.csv")    # Base per al model de freqüència
df_sev   = pd.read_csv("data/processed/df_sev_base.csv")     # Base per al model de severitat
df_ratio = pd.read_csv("data/processed/df_ratio_base.csv")   # Base per al model de ràtio econòmica

# Verifico ràpidament dimensions per assegurar-me que tot ha carregat bé
print("Freq:", df_freq.shape, "| Sev:", df_sev.shape, "| Ratio:", df_ratio.shape)

# ------------------------------------------------------------
# 1) Assegurar que totes les variables clau són numèriques
# ------------------------------------------------------------
# Definisc una funció auxiliar per forçar conversió a numèric.
# Qualsevol valor que no es pugui convertir passa a NaN, i deixo
# constància si la conversió genera nuls nous.
def ensure_numeric(df, cols, name="df"):
    """
    Força la conversió de les columnes indicades a tipus numèric.
    Si algun valor no és convertible, es transforma a NaN.
    També informo si la conversió crea NaN nous.
    """
    for col in cols:
        if col in df.columns:
            before_nulls = df[col].isna().sum()
            df[col] = pd.to_numeric(df[col], errors="coerce")
            after_nulls = df[col].isna().sum()
            new_nans = after_nulls - before_nulls
            if new_nans > 0:
                print(f"{name} — La columna '{col}' ha generat {new_nans} NaN nous en convertir a numèrica.")
    return df

# Llista de variables que vull garantir com a numèriques
numeric_cols = [
    "Driver_age",                # Edat del conductor
    "Licence_age",               # Antiguitat del carnet
    "Vehicle_age",               # Antiguitat del vehicle
    "Power",                     # Potència del vehicle
    "Value_vehicle",             # Valor assegurat del vehicle
    "Premium",                   # Prima anual
    "Cylinder_capacity",         # Cilindrada
    "Weight",                    # Pes del vehicle
    "Length",                    # Longitud del vehicle
    "Cost_claims_year",          # Cost anual de sinistres
    "Claims_to_premium_ratio"    # Ràtio cost/prima
]

# Aplico la conversió als tres datasets
df_freq  = ensure_numeric(df_freq,  numeric_cols, name="df_freq")
df_sev   = ensure_numeric(df_sev,   numeric_cols, name="df_sev")
df_ratio = ensure_numeric(df_ratio, numeric_cols, name="df_ratio")

print("\nConversió a tipus numèric aplicada quan calia.")

# ------------------------------------------------------------
# 2) Transformacions logarítmiques (log1p)
# ------------------------------------------------------------
# Aplico transformacions logarítmiques a variables clarament asimètriques,
# utilitzant log(1 + x) per evitar problemes amb zeros.
log_vars = ["Cost_claims_year", "Value_vehicle", "Power"]

for col in log_vars:
    if col in df_freq.columns:
        df_freq[col + "_log"] = np.log1p(df_freq[col])
    if col in df_sev.columns:
        df_sev[col + "_log"] = np.log1p(df_sev[col])

print("Transformacions logarítmiques aplicades a:", log_vars)

# ------------------------------------------------------------
# 3) Winsorització / Capping
# ------------------------------------------------------------
# Definisc una funció de capping conservador basada en percentils.
# No elimino registres: només limito els valors extrems.
def winsorize_series(s, lower=0.01, upper=0.99):
    """
    Aplica capping basat en percentils 1% i 99%.
    Serveix per reduir l’impacte d’outliers molt extrems.
    """
    lo = s.quantile(lower)
    hi = s.quantile(upper)
    return np.clip(s, lo, hi)

# Variables on vull aplicar capping
winsor_vars = ["Value_vehicle", "Power", "Premium", "Cost_claims_year"]

for col in winsor_vars:
    if col in df_freq.columns:
        df_freq[col + "_cap"] = winsorize_series(df_freq[col])
    if col in df_sev.columns:
        df_sev[col + "_cap"] = winsorize_series(df_sev[col])

print("Capping aplicat a:", winsor_vars)

# ------------------------------------------------------------
# 4) Escalat per a models basats en gradient
# ------------------------------------------------------------
# Estandarditzo algunes variables capejades per facilitar
# la convergència de models com GBM, XGBoost, etc.
scaler = StandardScaler()

scale_vars = ["Value_vehicle_cap", "Power_cap", "Premium_cap"]

for col in scale_vars:
    if col in df_freq.columns:
        df_freq[col + "_scaled"] = scaler.fit_transform(df_freq[[col]])
    if col in df_sev.columns:
        df_sev[col + "_scaled"] = scaler.fit_transform(df_sev[[col]])

print("Estandardització (z-score) aplicada quan calia.")

# ------------------------------------------------------------
# 5) Binning per a models lineals / GLM
# ------------------------------------------------------------
# Discretitzo algunes variables contínues en quantils
# per reforçar relacions més lineals en models tipus GLM.
bin_vars = ["Driver_age", "Vehicle_age", "Power", "Value_vehicle"]

for col in bin_vars:
    if col in df_freq.columns:
        kb = KBinsDiscretizer(
            n_bins=6,
            encode="ordinal",
            strategy="quantile",
            quantile_method="linear"
        )
        df_freq[col + "_bin"] = kb.fit_transform(df_freq[[col]])

    if col in df_sev.columns:
        kb = KBinsDiscretizer(
            n_bins=6,
            encode="ordinal",
            strategy="quantile",
            quantile_method="linear"
        )
        df_sev[col + "_bin"] = kb.fit_transform(df_sev[[col]])

print("Binning aplicat a:", bin_vars)

# ------------------------------------------------------------
# 6) Imputació final simple (NaN → -1)
# ------------------------------------------------------------
# Com a últim pas defensiu, substitueixo qualsevol NaN residual
# per -1. No és una imputació informativa, només evita errors
# en models de Machine Learning.
df_freq  = df_freq.fillna(-1)
df_sev   = df_sev.fillna(-1)
df_ratio = df_ratio.fillna(-1)

print("Imputació final completada (NaN → -1).")

# ------------------------------------------------------------
# 7) Guardar datasets transformats
# ------------------------------------------------------------
# Deso els datasets ja transformats i llestos per a la fase
# de modelització.
df_freq.to_csv("data/processed/df_freq_num_transformed.csv", index=False, encoding="utf-8")
df_sev.to_csv("data/processed/df_sev_num_transformed.csv", index=False, encoding="utf-8")
df_ratio.to_csv("data/processed/df_ratio_num_transformed.csv", index=False, encoding="utf-8")

print("Datasets transformats guardats correctament a data/processed/")


Freq: (105555, 24) | Sev: (19646, 18) | Ratio: (105555, 24)

Cast a float/int aplicat quan calia.
Transformacions log1p aplicades: ['Cost_claims_year', 'Value_vehicle', 'Power']
Capping aplicat a: ['Value_vehicle', 'Power', 'Premium', 'Cost_claims_year']
Escalat (z-score) aplicat quan calia.
Binning aplicat a: ['Driver_age', 'Vehicle_age', 'Power', 'Value_vehicle']
Imputació simple final completada (valors NA → -1).
Datasets transformats desats a data/processed/


#### <b>3.3.4.3 Transformació i codificació de variables categòriques</b>

In [10]:
# ============================================================
# 3.3.4.3 TRANSFORMACIÓ I CODIFICACIÓ DE VARIABLES CATEGÒRIQUES
# ============================================================
# En aquest script preparo les variables categòriques perquè les pugui
# fer servir en diferents famílies de models, mantenint sempre la regla
# més important: no contaminar el test (2018) amb informació del train.
#
# El que faig aquí és:
#   - Preparar categòriques per a:
#       (A) GLM/GAM  -> One-Hot Encoding (fit només amb TRAIN)
#       (B) GBM      -> Target Encoding (calculat només amb TRAIN)
#   - Aplicar codificació ordinal a Type_risk (ordre natural 1<2<3<4)
#   - Tenir el dataset exportat en formats separats segons el model
#
# Notes crítiques que m'asseguro de respectar:
#   - Tant OHE com Target Encoding els fitjo/estimo només amb TRAIN.
#   - Mai calculo target encoding amb tot el dataset perquè això seria leakage temporal.
# ============================================================

import os
import numpy as np
import pandas as pd

from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder

# Creo la carpeta de sortida si no existeix
os.makedirs("data/processed", exist_ok=True)

# ------------------------------------------------------------
# 0) Càrrega dels datasets numèrics transformats (3.3.4.2)
# ------------------------------------------------------------
# Aquí carrego el dataset de freqüència ja amb transformacions numèriques aplicades.
# Els datasets de severitat i ràtio els assumisc ja carregats d'abans (seguint el pipeline),
# però igualment els valido després.
df_freq  = pd.read_csv("data/processed/df_freq_num_transformed.csv")

print("Data carregada:")
print("Freq:", df_freq.shape, "| Sev:", df_sev.shape, "| Ratio:", df_ratio.shape)

# Validacions mínimes per assegurar-me que puc fer split per train/test
for df_, name in [(df_freq, "df_freq"), (df_sev, "df_sev"), (df_ratio, "df_ratio")]:
    if "set_type" not in df_.columns:
        raise ValueError(f"{name} no conté 'set_type'. Revisa 3.3.4.1/3.3.4.2.")
    if "Policy_year" not in df_.columns:
        raise ValueError(f"{name} no conté 'Policy_year'. Revisa 3.3.4.1/3.3.4.2.")

# Target de freqüència (necessari per al target encoding)
target_freq = "Has_claims_year"

# ------------------------------------------------------------
# 1) Llistes de variables categòriques
# ------------------------------------------------------------
# Defineixo quines variables vull tractar com a categòriques.
# Després em quedo només amb les que realment existeixen al dataframe.
cat_vars = [
    "Type_risk",
    "Area",
    "Type_fuel",
    "Distribution_channel",
    "Second_driver",
    "Has_lapse",
    "Has_claims_history",
]

cat_vars = [c for c in cat_vars if c in df_freq.columns]
print("Variables categòriques detectades (df_freq):", cat_vars)

# Per coherència, converteixo les categòriques a string abans d'entrar a encoders.
# Això evita barreges de tipus (p.ex. 1 vs "1") i fa que l'OHE/ordinal sigui estable.
def cast_cats_to_str(df, cols):
    for c in cols:
        if c in df.columns:
            df[c] = df[c].astype(str)
    return df

df_freq  = cast_cats_to_str(df_freq, cat_vars)
df_sev   = cast_cats_to_str(df_sev,  [c for c in cat_vars if c in df_sev.columns])
df_ratio = cast_cats_to_str(df_ratio,[c for c in cat_vars if c in df_ratio.columns])

# ------------------------------------------------------------
# 3) Codificació ordinal (Type_risk) — FIT només TRAIN
# ------------------------------------------------------------
# Type_risk té un ordre natural (1<2<3<4), així que aquí m'interessa
# capturar-ho amb una codificació ordinal.
# Si apareix alguna etiqueta fora d'aquest conjunt, la codifico com -1.
if "Type_risk" in df_freq.columns:
    ord_enc = OrdinalEncoder(
        categories=[["1", "2", "3", "4"]],
        handle_unknown="use_encoded_value",
        unknown_value=-1
    )

    # Defineixo el train mask per freq (és el que fa de referència per fit)
    train_mask_freq = df_freq["set_type"].astype(str) == "train"

    # Fit només amb TRAIN per evitar leakage temporal
    ord_enc.fit(df_freq.loc[train_mask_freq, ["Type_risk"]])

    # Transformo tot el dataset de freq (train+test) amb l'encoder ja fitjat
    df_freq["Type_risk_ord"] = ord_enc.transform(df_freq[["Type_risk"]])

    # Reutilitzo el mateix encoder per sev i ratio (mateix mapping, fitjat amb train de freq)
    if "Type_risk" in df_sev.columns:
        df_sev["Type_risk_ord"] = ord_enc.transform(df_sev[["Type_risk"]])
    if "Type_risk" in df_ratio.columns:
        df_ratio["Type_risk_ord"] = ord_enc.transform(df_ratio[["Type_risk"]])

print("Codificació ordinal aplicada a Type_risk.")

# ------------------------------------------------------------
# 4) One-Hot Encoding per GLM/GAM — FIT només TRAIN, TRANSFORM a tot
# ------------------------------------------------------------
# Per a models lineals/GLM/GAM, el que necessito és una matriu de dummies.
# Aquí faig fit només amb TRAIN per definir l'espai de categories.
glm_cat_vars = cat_vars.copy()

# Em preparo un OneHotEncoder compatible amb versions diferents de sklearn
try:
    ohe = OneHotEncoder(handle_unknown="ignore", sparse_output=False)
except TypeError:
    ohe = OneHotEncoder(handle_unknown="ignore", sparse=False)

# Treballo sobre una còpia per separar clarament "dataset GLM"
df_glm = df_freq.copy()

train_mask = df_glm["set_type"].astype(str) == "train"

# Fit només amb TRAIN (categories definides amb dades fins 2017)
ohe.fit(df_glm.loc[train_mask, glm_cat_vars])

# Transformo tot (train+test) amb el mateix encoder (categories no vistes → tot a 0)
glm_encoded = ohe.transform(df_glm[glm_cat_vars])
ohe_cols = ohe.get_feature_names_out(glm_cat_vars)

# Construeixo DataFrame amb les dummies i el concateno
df_ohe = pd.DataFrame(glm_encoded, columns=ohe_cols, index=df_glm.index)
df_glm = pd.concat([df_glm.drop(columns=glm_cat_vars), df_ohe], axis=1)

print("One-Hot Encoding aplicat (dataset GLM/GAM).")
print("Dimensions df_glm:", df_glm.shape)

# ------------------------------------------------------------
# 5) Target Encoding per GBM — calculat només amb TRAIN, aplicat a tot
# ------------------------------------------------------------
# Per models tipus GBM, un target encoding pot ser útil per capturar
# informació agregada de categories sense explotar dimensionalitat.
# Sempre el calculo NOMÉS amb TRAIN.
def target_encode_train_only(df, col, target, set_col="set_type"):
    """
    Target encoding simple:
      - calcula la mitjana del target per categoria amb TRAIN
      - aplica al dataset complet
      - categories no vistes -> global mean (TRAIN)
    """
    train_mask = df[set_col].astype(str) == "train"
    global_mean = df.loc[train_mask, target].mean()

    means = df.loc[train_mask].groupby(col)[target].mean()
    encoded = df[col].map(means)

    return encoded.fillna(global_mean)

# Creo el dataset específic per GBM sobre una còpia del df_freq
df_gbm = df_freq.copy()

# Aplico target encoding a totes les variables categòriques definides
# (incloent binàries, tot i que aquí és una decisió més pràctica que teòrica).
for col in cat_vars:
    df_gbm[col + "_te"] = target_encode_train_only(df_gbm, col, target_freq, set_col="set_type")

# Un cop tinc les versions _te, puc eliminar les categòriques originals
df_gbm = df_gbm.drop(columns=cat_vars)

print("Target Encoding aplicat (dataset GBM).")
print("Dimensions df_gbm:", df_gbm.shape)

# ------------------------------------------------------------
# 6) Guardar datasets finals (freq)
# ------------------------------------------------------------
# Exporto els dos datasets finals per freqüència:
#   - df_glm: per GLM/GAM amb dummies
#   - df_gbm: per GBM amb target encoding
df_glm.to_csv("data/processed/df_freq_cat_glm.csv", index=False, encoding="utf-8")
df_gbm.to_csv("data/processed/df_freq_cat_gbm.csv", index=False, encoding="utf-8")

print("Datasets categòrics transformats i desats correctament:")
print(" - data/processed/df_freq_cat_glm.csv")
print(" - data/processed/df_freq_cat_gbm.csv")


Data carregada:
Freq: (105555, 36) | Sev: (19646, 31) | Ratio: (105555, 24)
Variables categòriques detectades (df_freq): ['Type_risk', 'Area', 'Type_fuel', 'Distribution_channel', 'Second_driver', 'Has_lapse', 'Has_claims_history']
Codificació ordinal aplicada a Type_risk.
One-Hot Encoding aplicat (dataset GLM/GAM).
Dimensions df_glm: (105555, 47)
Target Encoding aplicat (dataset GBM).
Dimensions df_gbm: (105555, 37)
Datasets categòrics transformats i desats correctament:
 - data/processed/df_freq_cat_glm.csv
 - data/processed/df_freq_cat_gbm.csv


#### <b>3.3.4.4	Variables derivades i indicadors compostos</b>

In [13]:
# ============================================================
# 3.3.4.4 VARIABLES DERIVADES I INDICADORS COMPOSTOS
# ============================================================
# En aquest bloc faig un feature engineering més "de negoci" i d'interaccions,
# amb l’objectiu de capturar efectes combinats que a l’EDA ja s’intuïen,
# però sense caure en una explosió dimensional.
#
# El que busco aquí és:
#   - Relacions conductor × vehicle (interaccions numèriques clares)
#   - Ràtios econòmics derivats (prima/valor, valor/potència, etc.)
#   - Algunes interaccions amb categòriques ja codificades (GLM: dummies, GBM: target encoding)
#   - Mantenir-ho "selectiu": poques interaccions però amb sentit actuarial
#   - Exportar datasets ja enriquits per passar a 3.3.5 (partició i modelatge)
#
# Nota:
#   - Parteixo de datasets ja transformats:
#       * df_freq_cat_glm.csv            → freq per GLM/GAM (OHE)
#       * df_freq_cat_gbm.csv            → freq per GBM (target encoding)
#       * df_sev_num_transformed.csv     → severitat numèrica transformada
# ============================================================

# Opcions de display per inspecció en consola
pd.set_option("display.max_columns", 200)
pd.set_option("display.width", 140)

# ------------------------------------------------------------
# 0) Càrrega de datasets transformats previs
# ------------------------------------------------------------
# Carrego els fitxers generats a passos anteriors i verifico dimensions
# per assegurar-me que estic treballant amb el que toca.
freq_glm_path = "data/processed/df_freq_cat_glm.csv"          # Freqüència per GLM/GAM
freq_gbm_path = "data/processed/df_freq_cat_gbm.csv"          # Freqüència per GBM (target encoding)
sev_num_path  = "data/processed/df_sev_num_transformed.csv"   # Severitat numèrica

df_glm = pd.read_csv(freq_glm_path)
df_gbm = pd.read_csv(freq_gbm_path)
df_sev = pd.read_csv(sev_num_path)

print("Freq GLM:", df_glm.shape)
print("Freq GBM:", df_gbm.shape)
print("Severitat:", df_sev.shape)

# ------------------------------------------------------------
# 1) Helpers per crear interaccions i ràtios de forma segura
# ------------------------------------------------------------
# Em creo dues funcions petites per:
#   - fer interaccions (multiplicacions) sense petar si falta alguna columna
#   - crear ràtios amb control de divisió per zero (epsilon al denominador)
def add_interaction(df, col_a, col_b, new_name=None):
    """
    Afegeixo una columna d’interacció = df[col_a] * df[col_b]
    només si existeixen les dues columnes al dataframe.
    """
    if col_a in df.columns and col_b in df.columns:
        if new_name is None:
            new_name = f"{col_a}_x_{col_b}"
        df[new_name] = df[col_a] * df[col_b]
        print(f"   ➜ Creat: {new_name}")
    else:
        missing = [c for c in [col_a, col_b] if c not in df.columns]
        print(f"   ⚠ No s'ha creat interacció {col_a}×{col_b} (manca: {missing})")
    return df

def safe_ratio(df, num_col, den_col, new_name):
    """
    Creo un ràtio num_col / den_col de manera segura:
      - només si existeixen les columnes
      - afegeixo un epsilon petit al denominador per evitar divisió per 0
    """
    if num_col in df.columns and den_col in df.columns:
        df[new_name] = df[num_col] / (df[den_col] + 1e-6)
        print(f"   ➜ Creat ràtio: {new_name}")
    else:
        missing = [c for c in [num_col, den_col] if c not in df.columns]
        print(f"   No s'ha creat ràtio {new_name} (manca: {missing})")
    return df

# ------------------------------------------------------------
# 2) Interaccions numèriques de risc combinat (freqüència)
# ------------------------------------------------------------
# Aquí creo interaccions i ràtios que tenen sentit actuarial:
#   - combinacions conductor/vehicle
#   - proxies econòmiques simples
print("\n[Freqüència - GLM] Interaccions numèriques principals:")

# Edat conductor × potència (proxy: perfil del conductor en vehicles més potents)
df_glm = add_interaction(df_glm, "Driver_age", "Power", "Driver_age_x_Power")

# Antiguitat vehicle × valor vehicle (proxy: valor residual + envelliment)
df_glm = add_interaction(df_glm, "Vehicle_age", "Value_vehicle", "Vehicle_age_x_Value_vehicle")

# Valor / potència (proxy: “valor per unitat de potència”)
df_glm = safe_ratio(df_glm, "Value_vehicle", "Power", "Value_to_power")

# Prima / valor (proxy: intensitat de prima vs capital cobert)
df_glm = safe_ratio(df_glm, "Premium", "Value_vehicle", "Premium_to_value")

print("\n[Freqüència - GBM] Interaccions numèriques principals:")

df_gbm = add_interaction(df_gbm, "Driver_age", "Power", "Driver_age_x_Power")
df_gbm = add_interaction(df_gbm, "Vehicle_age", "Value_vehicle", "Vehicle_age_x_Value_vehicle")

df_gbm = safe_ratio(df_gbm, "Value_vehicle", "Power", "Value_to_power")
df_gbm = safe_ratio(df_gbm, "Premium", "Value_vehicle", "Premium_to_value")

# ------------------------------------------------------------
# 3) Interaccions amb variables categòriques codificades
# ------------------------------------------------------------
# Aquí faig interaccions "selectives" amb categòriques ja codificades.
#   - GLM: Type_risk_ord × dummies d’Area
#   - GLM: dummies de Second_driver × Power
#   - GBM: interaccions entre target-encoded (ja són numèriques)
print("\n[Freqüència - GLM] Interaccions Type_risk × Area:")

if "Type_risk_ord" in df_glm.columns:
    area_cols = [c for c in df_glm.columns if c.startswith("Area_")]
    for ac in area_cols:
        new_name = f"Type_risk_ord_x_{ac}"
        df_glm[new_name] = df_glm["Type_risk_ord"] * df_glm[ac]
        print(f"   ➜ Creat: {new_name}")
else:
    print("   No hi ha Type_risk_ord a df_glm.")

print("\n[Freqüència - GLM] Interaccions Second_driver × Power:")

# Interacció entre dummies de Second_driver i potència
power_col = "Power"
second_driver_cols = [c for c in df_glm.columns if c.startswith("Second_driver_")]

for sd in second_driver_cols:
    new_name = f"{sd}_x_{power_col}"
    if power_col in df_glm.columns:
        df_glm[new_name] = df_glm[sd] * df_glm[power_col]
        print(f"   ➜ Creat: {new_name}")
    else:
        print(f"   ⚠ No s'ha creat {new_name} (manca Power).")

print("\n[Freqüència - GBM] Interaccions categòriques resumides:")

# En GBM, les variables _te ja són numèriques, així que puc fer interaccions directes.
# Type_risk_te × Area_te
if "Type_risk_te" in df_gbm.columns and "Area_te" in df_gbm.columns:
    df_gbm["Type_risk_te_x_Area_te"] = df_gbm["Type_risk_te"] * df_gbm["Area_te"]
    print("   ➜ Creat: Type_risk_te_x_Area_te")

# Second_driver_te × Power
if "Second_driver_te" in df_gbm.columns and "Power" in df_gbm.columns:
    df_gbm["Second_driver_te_x_Power"] = df_gbm["Second_driver_te"] * df_gbm["Power"]
    print("   ➜ Creat: Second_driver_te_x_Power")

# ------------------------------------------------------------
# 4) Variables derivades per severitat
# ------------------------------------------------------------
# Aquí treballo amb df_sev numèric (sense OHE) i creo variables
# útils per capturar efectes d’escala i intensitat econòmica.
print("\n[Severitat] Variables derivades i interaccions:")

# Antiguitat vehicle × valor vehicle
df_sev = add_interaction(df_sev, "Vehicle_age", "Value_vehicle", "Vehicle_age_x_Value_vehicle")

# Potència × valor vehicle (proxy: segment “vehicle potent i car”)
df_sev = add_interaction(df_sev, "Power", "Value_vehicle", "Power_x_Value_vehicle")

# Cost / prima (intensitat del cost respecte prima)
df_sev = safe_ratio(df_sev, "Cost_claims_year", "Premium", "Sev_cost_to_premium")

# Valor / prima (capital cobert vs prima)
df_sev = safe_ratio(df_sev, "Value_vehicle", "Premium", "Sev_value_to_premium")

# Interacció simple Type_risk × Area (si existeixen i té sentit numèric)
# Ho deixo com a diagnòstic/interacció simple perquè és una combinació molt "de negoci".
if "Type_risk" in df_sev.columns and "Area" in df_sev.columns:
    try:
        df_sev["Type_risk"] = pd.to_numeric(df_sev["Type_risk"], errors="coerce")
        df_sev["Type_risk_x_Area"] = df_sev["Type_risk"] * df_sev["Area"]
        print("   ➜ Creat: Type_risk_x_Area")
    except Exception as e:
        print("No s'ha pogut crear Type_risk_x_Area:", e)

# ------------------------------------------------------------
# 5) Desar datasets enriquits
# ------------------------------------------------------------
# Exporto els datasets amb el feature engineering aplicat
# perquè quedin llestos per a partició i modelatge (3.3.5).
os.makedirs("data/processed", exist_ok=True)

freq_glm_fe_path = "data/processed/df_freq_fe_glm.csv"
freq_gbm_fe_path = "data/processed/df_freq_fe_gbm.csv"
sev_fe_path      = "data/processed/df_sev_fe.csv"

df_glm.to_csv(freq_glm_fe_path, index=False)
df_gbm.to_csv(freq_gbm_fe_path, index=False)
df_sev.to_csv(sev_fe_path, index=False)

print("\nDatasets enriquits desats:")
print("   -", freq_glm_fe_path)
print("   -", freq_gbm_fe_path)
print("   -", sev_fe_path)

print("\n3.3.4.4 complet - Variables derivades i interaccions creades correctament.")


Freq GLM: (105555, 47)
Freq GBM: (105555, 37)
Severitat: (19646, 31)

[Freqüència - GLM] Interaccions numèriques principals:
   ➜ Creat: Driver_age_x_Power
   ➜ Creat: Vehicle_age_x_Value_vehicle
   ➜ Creat ràtio: Value_to_power
   ➜ Creat ràtio: Premium_to_value

[Freqüència - GBM] Interaccions numèriques principals:
   ➜ Creat: Driver_age_x_Power
   ➜ Creat: Vehicle_age_x_Value_vehicle
   ➜ Creat ràtio: Value_to_power
   ➜ Creat ràtio: Premium_to_value

[Freqüència - GLM] Interaccions Type_risk × Area:
   ➜ Creat: Type_risk_ord_x_Area_0
   ➜ Creat: Type_risk_ord_x_Area_1

[Freqüència - GLM] Interaccions Second_driver × Power:
   ➜ Creat: Second_driver_0_x_Power
   ➜ Creat: Second_driver_1_x_Power

[Freqüència - GBM] Interaccions categòriques resumides:
   ➜ Creat: Type_risk_te_x_Area_te
   ➜ Creat: Second_driver_te_x_Power

[Severitat] Variables derivades i interaccions:
   ➜ Creat: Vehicle_age_x_Value_vehicle
   ➜ Creat: Power_x_Value_vehicle
   ➜ Creat ràtio: Sev_cost_to_premium
  

#### <b>3.3.4.5 Reducció de dimensionalitat i anàlisi multivariable</b>

In [16]:
# ============================================================
# 3.3.4.5 REDUCCIÓ DE DIMENSIONALITAT I ANÀLISI MULTIVARIANT
# ============================================================
# En aquest script faig un pas de diagnòstic multivariant per entendre
# millor la redundància entre predictors i reduir problemes típics en
# models lineals (sobretot GLM amb moltes dummies/interaccions).
#
# El que vull aconseguir és:
#   - Mesurar multicol·linearitat (VIF) i correlacions altes al dataset GLM
#   - Detectar variables redundants i candidates a eliminar en el GLM
#   - Fer un PCA exploratori al dataset GBM només com a diagnòstic
#   - Entrenar un Gradient Boosting bàsic per obtenir importàncies
#   - Exportar fitxers de suport i una llista de variables "drop candidates"
# ============================================================

import pandas as pd
import numpy as np
import warnings
from sklearn.decomposition import PCA
from sklearn.ensemble import GradientBoostingClassifier
from statsmodels.stats.outliers_influence import variance_inflation_factor

# ------------------------------------------------------------
# 0) Carregar datasets enriquits
# ------------------------------------------------------------
# Aquí carrego els datasets sortints de 3.3.4.4:
#   - df_freq_fe_glm: freqüència per GLM (OHE + interaccions)
#   - df_freq_fe_gbm: freqüència per GBM (target encoding + interaccions)
# El dataset de severitat no el faig servir en càlculs aquí, però el
# mantinc present al pipeline (per coherència global).
df_glm = pd.read_csv("data/processed/df_freq_fe_glm.csv")
df_gbm = pd.read_csv("data/processed/df_freq_fe_gbm.csv")

print("Carregats:")
print("GLM:", df_glm.shape)
print("GBM:", df_gbm.shape)
print("SEV:", df_sev.shape)

# ------------------------------------------------------------
# 1) Seleccionar només variables numèriques
# ------------------------------------------------------------
# Per calcular VIF, correlacions i PCA necessito treballar només amb numèriques.
def get_numeric(df):
    """Retorno només les columnes numèriques (int/float) del DataFrame."""
    return df.select_dtypes(include=[np.number])

num_glm = get_numeric(df_glm)

# ------------------------------------------------------------
# 1.1 Eliminar columnes constants o duplicades (abans del VIF)
# ------------------------------------------------------------
# Abans del VIF elimino:
#   - columnes constants (no aporten informació i poden trencar càlculs)
#   - columnes duplicades (exactament iguals, generen singularitat)
constant_cols = [c for c in num_glm.columns if num_glm[c].nunique() <= 1]
duplicated_cols = num_glm.T[num_glm.T.duplicated()].index.tolist()

cols_to_remove = set(constant_cols + duplicated_cols)

if len(cols_to_remove) > 0:
    print("Columnes eliminades abans del VIF (constants/duplicades):")
    print(cols_to_remove)

# Creo el dataframe net per a VIF i correlacions
num_glm_clean = num_glm.drop(columns=list(cols_to_remove), errors="ignore")

# ------------------------------------------------------------
# 2) Càlcul de VIF amb control d’errors
# ------------------------------------------------------------
# El VIF mesura multicol·linearitat. En general, un VIF alt indica que
# una variable es pot predir molt bé amb la resta (redundància).
def compute_vif(df):
    """
    Calculo el VIF per a totes les variables numèriques.
    Excloc la variable objectiu si existeix, perquè no té sentit fer VIF del target.
    """
    vif_data = []

    # X = predictors numèrics (sense target)
    X = df.drop(columns=["Has_claims_year"], errors="ignore").copy()

    # Torno a eliminar columnes constants per seguretat
    const_cols = X.columns[X.std() == 0]
    if len(const_cols) > 0:
        X = X.drop(columns=const_cols)

    # Calculo VIF variable a variable, controlant warnings típics
    for i in range(X.shape[1]):
        with warnings.catch_warnings():
            warnings.filterwarnings("ignore", category=RuntimeWarning)
            vif = variance_inflation_factor(X.values, i)
        vif_data.append((X.columns[i], vif))

    vif_df = pd.DataFrame(vif_data, columns=["Variable", "VIF"])
    # Si algun VIF surt infinit, el converteixo a NaN per no arrossegar problemes després
    vif_df["VIF"] = vif_df["VIF"].replace([np.inf, -np.inf], np.nan)
    return vif_df

# Aplico VIF al subconjunt numèric net
vif_glm = compute_vif(num_glm_clean)

# Desa a disc per traçabilitat i revisió posterior
vif_glm.to_csv("data/processed/vif_glm.csv", index=False)
print("VIF calculat per GLM i guardat a data/processed/vif_glm.csv.")

# ------------------------------------------------------------
# 3) Correlacions altes > |0.75|
# ------------------------------------------------------------
# Complemento el VIF amb correlacions absolutes altes entre parelles,
# perquè això em dona una lectura més directa de redundància bivariant.
corr_glm = num_glm_clean.corr().abs()

# Agafo només la meitat superior de la matriu (evito duplicats i diagonal)
high_corr_pairs = (
    corr_glm.where(np.triu(np.ones(corr_glm.shape), k=1).astype(bool))
    .stack()
    .reset_index()
)
high_corr_pairs.columns = ["Var1", "Var2", "Correlation"]

# Filtre per correlació alta (primer llindar més permissiu)
high_corr_pairs = high_corr_pairs[high_corr_pairs["Correlation"] > 0.75]

# Deso parelles correlacionades per poder inspeccionar-ho amb calma
high_corr_pairs.to_csv("data/processed/high_corr_pairs_glm.csv", index=False)
print("Parelles amb correlació alta identificades i desades a high_corr_pairs_glm.csv.")

# ------------------------------------------------------------
# 4) PCA exploratori sobre GBM
# ------------------------------------------------------------
# Aquí faig PCA només com a diagnòstic per veure quanta variància
# capturen els primers components. No ho faig servir com a feature
# directe (si ho decideixo més endavant, ja ho justificaré).
num_gbm = get_numeric(df_gbm)

# X_gbm = predictors numèrics (sense target)
X_gbm = num_gbm.drop(columns=["Has_claims_year"], errors="ignore")

pca = PCA(n_components=5)
pca.fit(X_gbm)

pca_variance = pd.DataFrame({
    "Component": np.arange(1, 6),
    "Explained_variance": pca.explained_variance_ratio_
})
pca_variance.to_csv("data/processed/pca_variance_gbm.csv", index=False)

print("PCA exploratori completat i variàncies guardades a pca_variance_gbm.csv.")

# ------------------------------------------------------------
# 5) Importància de variables amb Gradient Boosting (GBM)
# ------------------------------------------------------------
# Entreno un GradientBoostingClassifier bàsic només com a diagnòstic
# per veure quines variables aporten més informació al target.
gb = GradientBoostingClassifier()

X = X_gbm
y = df_gbm["Has_claims_year"]

gb.fit(X, y)

importances = pd.DataFrame({
    "Variable": X.columns,
    "Importance": gb.feature_importances_
}).sort_values("Importance", ascending=False)

importances.to_csv("data/processed/gbm_importances.csv", index=False)
print("Importàncies GBM calculades i desades a gbm_importances.csv.")

# ------------------------------------------------------------
# 6) Selecció final (regla simple per GLM)
# ------------------------------------------------------------
# Aquí creo una llista de variables candidates a eliminar al GLM.
# Ho faig amb una regla simple i transparent:
#   - VIF > 10  → multicol·linearitat alta
#   - corr > 0.85 (parelles molt correlacionades) → agafo Var2 com a candidata
vars_drop = list(
    vif_glm[vif_glm["VIF"] > 10]["Variable"].unique()
) + list(
    high_corr_pairs[high_corr_pairs["Correlation"] > 0.85]["Var2"].unique()
)

# Elimino duplicats perquè una variable pugui sortir per múltiples criteris
vars_drop = list(set(vars_drop))

# Deso la llista per tenir-la documentada i reutilitzable al pas de modelització
pd.Series(vars_drop).to_csv("data/processed/vars_to_drop_glm.csv", index=False)

print("Variables candidates a eliminació en GLM (VIF>10 o corr>0.85):")
print(vars_drop)

print("\nArxius exportats a data/processed/:")
print(" - vif_glm.csv                 (VIF per variable)")
print(" - high_corr_pairs_glm.csv    (parelles molt correlacionades)")
print(" - pca_variance_gbm.csv       (variància explicada per PCA en GBM)")
print(" - gbm_importances.csv        (importància de variables en GBM)")
print(" - vars_to_drop_glm.csv       (llista de variables a considerar eliminar)")

print("\n3.3.4.5 complet (reducció de dimensionalitat i anàlisi multivariant).")


Carregats:
GLM: (105555, 55)
GBM: (105555, 43)
SEV: (19646, 36)
VIF calculat per GLM i guardat a data/processed/vif_glm.csv.
Parelles amb correlació alta identificades i desades a high_corr_pairs_glm.csv.
PCA exploratori completat i variàncies guardades a pca_variance_gbm.csv.
Importàncies GBM calculades i desades a gbm_importances.csv.
Variables candidates a eliminació en GLM (VIF>10 o corr>0.85):
['Value_vehicle_log', 'Area_0', 'Has_lapse_1', 'Premium', 'Value_vehicle_bin', 'Type_risk_1', 'Type_risk_4', 'Second_driver_1', 'Power_log', 'Driver_age_bin', 'Has_claims_history_0', 'Type_risk_ord', 'Premium_cap', 'Type_fuel_D', 'Vehicle_age_bin', 'Area_1', 'Type_risk_2', 'Value_vehicle_cap', 'Driver_age_x_Power', 'Type_risk_ord_x_Area_0', 'Premium_cap_scaled', 'Second_driver_1_x_Power', 'Value_vehicle_cap_scaled', 'Driver_age', 'Distribution_channel_0', 'Power_cap_scaled', 'Type_risk_3', 'Type_risk_ord_x_Area_1', 'Type_fuel_P', 'Power_cap', 'Has_claims_history_1', 'Has_lapse_0', 'Value_veh

#### <b>3.3.4.6 Generació del dataset final per models</b>

In [19]:
# ============================================================
# 3.3.4.6 GENERACIÓ DEL DATASET FINAL PER A MODELS
# ============================================================
# En aquest pas consolido els datasets finals que aniran directament
# a la fase de modelització. L’objectiu és deixar-ho tot preparat,
# consistent i traçable:
#
#   - Carrego els datasets enriquits (freq GLM, freq GBM i severitat)
#   - Aplico la reducció de dimensionalitat decidida a 3.3.4.5 sobre el GLM
#     (VIF/correlacions), però protegint explícitament variables de segmentació
#     actuarial perquè no vull perdre els "talls" clau de cartera.
#   - Em garanteixo que metadades (ID, Policy_year, set_type) i targets hi són
#   - Separo el dataset de ràtio econòmica (si existeix) com a dataset independent
#   - M’asseguro que Claims_to_premium_ratio no entra com a predictor en freq/sev
#   - Exporto els CSV finals a data/model/
# ============================================================

import os
import pandas as pd

# Ajusto opcions de display per inspecció ràpida
pd.set_option("display.max_columns", 200)
pd.set_option("display.width", 140)

# ------------------------------------------------------------
# 0) Càrrega datasets enriquits
# ------------------------------------------------------------
# Carrego els datasets sortints de l'enginyeria de variables:
#   - freq GLM: OHE + interaccions
#   - freq GBM: target encoding + interaccions
#   - severitat: features derivades per al model de cost
freq_glm_path = "data/processed/df_freq_fe_glm.csv"
freq_gbm_path = "data/processed/df_freq_fe_gbm.csv"
sev_path      = "data/processed/df_sev_fe.csv"

df_freq_glm = pd.read_csv(freq_glm_path)
df_freq_gbm = pd.read_csv(freq_gbm_path)
df_sev      = pd.read_csv(sev_path)

print("Carregats datasets enriquits:")
print("  Freq GLM:", df_freq_glm.shape)
print("  Freq GBM:", df_freq_gbm.shape)
print("  Severitat:", df_sev.shape)

# Dataset de ràtio econòmica (si existeix en aquesta fase)
ratio_path = "data/processed/df_ratio_num_transformed.csv"
if os.path.exists(ratio_path):
    df_ratio = pd.read_csv(ratio_path)
    print("  Ràtio econòmica:", df_ratio.shape)
else:
    df_ratio = None
    print(
        "  No s'ha trobat df_ratio_num_transformed.csv. "
        "Es descarta dataset de ràtio en aquesta fase."
    )

# ------------------------------------------------------------
# 1) Camps clau i targets
# ------------------------------------------------------------
# Defineixo noms de camps clau per uniformitzar comprovacions.
id_col   = "ID"
year_col = "Policy_year"
set_col  = "set_type"

target_freq  = "Has_claims_year"
target_sev   = "Cost_claims_year"
target_ratio = "Claims_to_premium_ratio"

# ------------------------------------------------------------
# 2) Aplicar decisions de 3.3.4.5 (reducció GLM)
#    protegint variables de segmentació actuarial
# ------------------------------------------------------------
# Aquí aplico la llista de "drop candidates" del GLM, però amb una regla clara:
#   - mai elimino metadades ni targets
#   - mai elimino dummies/columnes que representen segmentació actuarial
#     (risc, àrea, combustible, canal, lapse, històric, segon conductor, etc.)
vars_drop_path = "data/processed/vars_to_drop_glm.csv"

if os.path.exists(vars_drop_path):
    # Carrego el fitxer com una sèrie (sense capçalera)
    vars_drop_series = pd.read_csv(vars_drop_path, header=None)[0]

    # Netejo entrades rares (p.ex. blancs, "0", etc.)
    vars_drop_raw = [
        v for v in vars_drop_series.tolist()
        if isinstance(v, str) and v.strip() != "" and v != "0"
    ]

    # Em quedo només amb les variables que realment existeixen al dataframe GLM
    vars_drop_in_glm = [v for v in vars_drop_raw if v in df_freq_glm.columns]

    print("\n[Selecció GLM] Variables candidates a eliminació segons 3.3.4.5:")
    print(f"  Total a vars_to_drop_glm.csv : {len(vars_drop_raw)}")
    print(f"  Presentes a df_freq_glm      : {len(vars_drop_in_glm)}")

    # Protegeixo explícitament metadades i targets
    protected_cols = {
        id_col, year_col, set_col,
        target_freq, target_sev
    }

    # També protegeixo variables de segmentació actuarial:
    # aquí assumeixo que, en GLM, aquestes variables apareixen com dummies amb prefixos.
    segmentation_prefixes = [
        "Type_risk_",
        "Area_",
        "Type_fuel_",
        "Distribution_channel_",
        "Second_driver_",
        "Has_lapse_",
        "Has_claims_history_",
    ]

    # Afegeixo a "protected_cols" qualsevol columna que comenci amb aquests prefixos
    for col in df_freq_glm.columns:
        if any(col.startswith(pref) for pref in segmentation_prefixes):
            protected_cols.add(col)

    # Finalment, només elimino el que està a la llista i NO és protegit
    vars_drop_final = [v for v in vars_drop_in_glm if v not in protected_cols]

    before_cols = df_freq_glm.shape[1]
    df_freq_glm = df_freq_glm.drop(columns=vars_drop_final, errors="ignore")
    after_cols = df_freq_glm.shape[1]

    print("\n[Selecció GLM] Resum de reducció de dimensionalitat:")
    print(f"  Columnes abans (df_freq_glm): {before_cols}")
    print(f"  Columnes després            : {after_cols}")
    print(f"  Nº de variables eliminades  : {len(vars_drop_final)}")

    if vars_drop_final:
        print("  Llista de variables eliminades en df_freq_glm:")
        print("   - " + "\n   - ".join(vars_drop_final))
    else:
        print("  No s'ha eliminat cap variable (totes protegides o no existents).")
else:
    print("\nAVÍS: No s'ha trobat 'vars_to_drop_glm.csv'.")
    print("   No s'aplica cap exclusió de variables al dataset GLM.")
    vars_drop_final = []

# ------------------------------------------------------------
# 2.bis) Assegurar que la ràtio econòmica NO entra al model GLM
# ------------------------------------------------------------
# Per coherència metodològica: Claims_to_premium_ratio és target d'un model
# de rendibilitat, però no l'he de fer servir com a predictor en freq/sev.
if target_ratio in df_freq_glm.columns:
    print(
        f"\n[Freq GLM] Eliminant {target_ratio} del dataset GLM "
        "(només s'utilitzarà com a target del model de ràtio)."
    )
    df_freq_glm = df_freq_glm.drop(columns=[target_ratio])

# ------------------------------------------------------------
# 3) Comprovacions de metadades i targets
# ------------------------------------------------------------
# Verifico que tots els datasets principals tenen metadades comunes
# (ID, Policy_year i set_type), perquè després el split sigui trivial.
for name, df_ in [
    ("Freq GLM", df_freq_glm),
    ("Freq GBM", df_freq_gbm),
    ("Severitat", df_sev),
]:
    missing = [c for c in [id_col, year_col, set_col] if c not in df_.columns]
    if missing:
        raise ValueError(f"Falta/n columna/es {missing} al dataset {name}.")

print("\nMetadades (ID, Policy_year, set_type) presents a tots els datasets principals.")

# Targets: aquí no aturo el pipeline si falten, però deixo avís explícit.
for name, df_, t in [
    ("Freq GLM", df_freq_glm, target_freq),
    ("Freq GBM", df_freq_gbm, target_freq),
    ("Severitat", df_sev, target_sev),
]:
    if t not in df_.columns:
        print(f"Avís: el target {t} no es troba al dataset {name}.")

# ------------------------------------------------------------
# 4) Resum temporal (sense split encara)
# ------------------------------------------------------------
# Només faig un check de distribució perquè em serveix per detectar
# errors ràpids (p.ex. tot ha quedat com a train per algun problema).
print("\nDistribució per set_type (Freq GLM):")
print(df_freq_glm[set_col].value_counts())

print("\nDistribució per set_type (Severitat):")
print(df_sev[set_col].value_counts())

# ------------------------------------------------------------
# 5) Desar datasets finals
# ------------------------------------------------------------
# Exporto els datasets finals consolidats a una carpeta "data/model".
# Encara no faig split train/test aquí; això ho faré explícitament al 3.3.6.
os.makedirs("data/model", exist_ok=True)

freq_glm_full_path = "data/model/freq_glm_full.csv"
freq_gbm_full_path = "data/model/freq_gbm_full.csv"
sev_full_path      = "data/model/sev_full.csv"

df_freq_glm.to_csv(freq_glm_full_path, index=False)
df_freq_gbm.to_csv(freq_gbm_full_path, index=False)
df_sev.to_csv(sev_full_path, index=False)

print("\nDatasets complets desats a:")
print(" -", freq_glm_full_path, "(freqüència GLM/GAM, amb reducció aplicada i sense Claims_to_premium_ratio)")
print(" -", freq_gbm_full_path, "(freqüència GBM)")
print(" -", sev_full_path,      "(severitat)")

# Dataset de ràtio econòmica separat (si existeix)
if df_ratio is not None:
    ratio_full_path = "data/model/ratio_full.csv"
    df_ratio.to_csv(ratio_full_path, index=False)
    print(" -", ratio_full_path, "(ràtio econòmica, amb Claims_to_premium_ratio com a target)")

print("\n3.3.4.6 complet - Datasets finals consolidats generats.")
print(" Les decisions de 3.3.4.5 (VIF/correlacions) s'han aplicat al dataset GLM,")
print(" però preservant explícitament les variables de segmentació actuarial clau.")
print(" Claims_to_premium_ratio s'utilitza només en el model de ràtio econòmica,")
print(" i no entra com a predictor en els models de freqüència ni severitat.")
print(" La divisió train/test (validació temporal) es realitzarà al punt 3.3.6.")


Carregats datasets enriquits:
  Freq GLM: (105555, 55)
  Freq GBM: (105555, 43)
  Severitat: (19646, 36)
  Ràtio econòmica: (105555, 24)

[Selecció GLM] Variables candidates a eliminació segons 3.3.4.5:
  Total a vars_to_drop_glm.csv : 35
  Presentes a df_freq_glm      : 35

[Selecció GLM] Resum de reducció de dimensionalitat:
  Columnes abans (df_freq_glm): 55
  Columnes després            : 39
  Nº de variables eliminades  : 16
  Llista de variables eliminades en df_freq_glm:
   - Value_vehicle_log
   - Premium
   - Value_vehicle_bin
   - Power_log
   - Driver_age_bin
   - Premium_cap
   - Vehicle_age_bin
   - Value_vehicle_cap
   - Driver_age_x_Power
   - Premium_cap_scaled
   - Value_vehicle_cap_scaled
   - Driver_age
   - Power_cap_scaled
   - Power_cap
   - Value_vehicle
   - Power_bin

Metadades (ID, Policy_year, set_type) presents a tots els datasets principals.

Distribució per set_type (Freq GLM):
set_type
train    69740
test     35815
Name: count, dtype: int64

Distribució p